# Chat Executor с вызовом функции

В этом блокноте рассматривается пример создания Chat Executor, использующего вызов функций.
Это полезно для быстрого начала работы.
Однако весьма вероятно, что вам захочется настроить логику — информацию об этом можно найти в других примерах в этой папке.

## Set up the chat model and tools

Here we will define the chat model and tools that we want to use.
Importantly, this model MUST support OpenAI function calling.

In [ ]:
%pip install gigachain gigachain_community duckduckgo-search

In [16]:
from langchain_community.chat_models import GigaChat
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage

Инициализируем модель и тул, которым может пользоваться агент.
В качестве тула используем поисковик duckduckgo. Также можно использовать поиск на базе API Tavily.

In [17]:
# tools = [TavilySearchResults(max_results=1)]
tools = [DuckDuckGoSearchRun()]
model = GigaChat(
    credentials=...,
    model=...functions_model...,
    verify_ssl_certs=False,
)

## Создаем исполнителя агентов

Теперь мы используем верхнеуровневый интерфейс для создания исполнителя

In [18]:
app = chat_agent_executor.create_function_calling_executor(model, tools)

Теперь мы можем вызвать исполнителя. На вход подается список сообщений в ключе `messages`.
В процессе исполнения видно, что агент обратился к поисковой системе, переформулирововав вопрос и затем сделал вывод из полученной информации.

In [19]:
inputs = {"messages": [HumanMessage(content="Нужно ли сегодня в Москве надевать шапку?")]}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'duckduckgo_search', 'arguments': {'query': 'погода в Москве сегодня'}}})]}
----
{'messages': [FunctionMessage(content='GISMETEO: Погода в Москве сейчас, текущая погода Москва, Москва (город федерального значения), Россия. 2 недели Радар Россия / Москва (город федерального значения) Погода в Москве сейчас Сейчас 17:17 −14 По ощущению −14 Сб, 10 фев Сегодня −17 −10 Вс, 11 фев Завтра −16 −8 Сб, 10 февраля, 17:17 8:06 Восход 17:21 Заход −14 По ощущению −14 Безоблачно Сегодня −7 −1 0,2 мм Вс, 4 фев Завтра −2 +2 3 мм 000 300 600 900 1200 1500 1800 2100 Температура воздуха, °C −3 −4 −6 −7 −3 −1 −1 −2 Порывы ветра, м/c Узнайте прогноз погоды в Москве на сегодня, завтра и на неделю. Смотрите советы по выживанию в снегопадах, метелях, гололедице и других ситуациях. Следите за актуальными новостями о погоде в столице и в других регионах России. Погода в Москве на сегодня, точный прогноз погоды в Москве на завтра и с